# UAS Teknologi Basis Data
- Nama: Muhammad Haerul
- NIM: H071201033
- Kelas: A 

## Create Engine

In [1]:
from sqlalchemy import create_engine
engine = create_engine('mysql://root@localhost/classicmodels', echo=False)

from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()
Base.metadata.create_all(engine)

from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind = engine)
session = Session()

## Create Models

In [2]:
from sqlalchemy import Column, DECIMAL, Date, ForeignKey, String, Text
from sqlalchemy.dialects.mysql import INTEGER, MEDIUMBLOB, MEDIUMTEXT, SMALLINT
from sqlalchemy.orm import relationship
from sqlalchemy.ext.declarative import declarative_base

Base = declarative_base()
metadata = Base.metadata

class Office(Base):
    __tablename__ = 'offices'

    officeCode = Column(String(10), primary_key=True)
    city = Column(String(50), nullable=False)
    phone = Column(String(50), nullable=False)
    addressLine1 = Column(String(50), nullable=False)
    addressLine2 = Column(String(50))
    state = Column(String(50))
    country = Column(String(50), nullable=False)
    postalCode = Column(String(15), nullable=False)
    territory = Column(String(10), nullable=False)


class Productline(Base):
    __tablename__ = 'productlines'

    productLine = Column(String(50), primary_key=True)
    textDescription = Column(String(4000))
    htmlDescription = Column(MEDIUMTEXT)
    image = Column(MEDIUMBLOB)


class Employee(Base):
    __tablename__ = 'employees'

    employeeNumber = Column(INTEGER(11), primary_key=True)
    lastName = Column(String(50), nullable=False)
    firstName = Column(String(50), nullable=False)
    extension = Column(String(10), nullable=False)
    email = Column(String(100), nullable=False)
    officeCode = Column(ForeignKey('offices.officeCode'), nullable=False, index=True)
    reportsTo = Column(ForeignKey('employees.employeeNumber'), index=True)
    jobTitle = Column(String(50), nullable=False)

    office = relationship('Office')
    parent = relationship('Employee', remote_side=[employeeNumber])


class Product(Base):
    __tablename__ = 'products'

    productCode = Column(String(15), primary_key=True)
    productName = Column(String(70), nullable=False)
    productLine = Column(ForeignKey('productlines.productLine'), nullable=False, index=True)
    productScale = Column(String(10), nullable=False)
    productVendor = Column(String(50), nullable=False)
    productDescription = Column(Text, nullable=False)
    quantityInStock = Column(SMALLINT(6), nullable=False)
    buyPrice = Column(DECIMAL(10, 2), nullable=False)
    MSRP = Column(DECIMAL(10, 2), nullable=False)

    productline = relationship('Productline')


class Customer(Base):
    __tablename__ = 'customers'

    customerNumber = Column(INTEGER(11), primary_key=True)
    customerName = Column(String(50), nullable=False)
    contactLastName = Column(String(50), nullable=False)
    contactFirstName = Column(String(50), nullable=False)
    phone = Column(String(50), nullable=False)
    addressLine1 = Column(String(50), nullable=False)
    addressLine2 = Column(String(50))
    city = Column(String(50), nullable=False)
    state = Column(String(50))
    postalCode = Column(String(15))
    country = Column(String(50), nullable=False)
    salesRepEmployeeNumber = Column(ForeignKey('employees.employeeNumber'), index=True)
    creditLimit = Column(DECIMAL(10, 2))

    employee = relationship('Employee')


class Order(Base):
    __tablename__ = 'orders'

    orderNumber = Column(INTEGER(11), primary_key=True)
    orderDate = Column(Date, nullable=False)
    requiredDate = Column(Date, nullable=False)
    shippedDate = Column(Date)
    status = Column(String(15), nullable=False)
    comments = Column(Text)
    customerNumber = Column(ForeignKey('customers.customerNumber'), nullable=False, index=True)

    customer = relationship('Customer')


class Payment(Base):
    __tablename__ = 'payments'

    customerNumber = Column(ForeignKey('customers.customerNumber'), primary_key=True, nullable=False)
    checkNumber = Column(String(50), primary_key=True, nullable=False)
    paymentDate = Column(Date, nullable=False)
    amount = Column(DECIMAL(10, 2), nullable=False)

    customer = relationship('Customer')


class Orderdetail(Base):
    __tablename__ = 'orderdetails'

    orderNumber = Column(ForeignKey('orders.orderNumber'), primary_key=True, nullable=False)
    productCode = Column(ForeignKey('products.productCode'), primary_key=True, nullable=False, index=True)
    quantityOrdered = Column(INTEGER(11), nullable=False)
    priceEach = Column(DECIMAL(10, 2), nullable=False)
    orderLineNumber = Column(SMALLINT(6), nullable=False)

    order = relationship('Order')
    product = relationship('Product')

## Define and Import Helper Functions

In [4]:
import pandas as pd
from sqlalchemy.sql import func
from sqlalchemy import or_


def to_df(query):
    df = pd.read_sql(query.statement, engine)
    return df

def show_df(query):
    df = pd.read_sql(query.statement, engine)
    return df

## 1

In [5]:
s = Session()
c = Customer
p = Product
o = Order
od = Orderdetail

num1 = s.query(c.customerName, p.productName).join(od.order).join(od.product).join(o.customer).filter(c.customerName == 'Herkku Gifts')
show_df(num1)

,customerName,productName
0,Herkku Gifts,1968 Ford Mustang
1,Herkku Gifts,1968 Dodge Charger
2,Herkku Gifts,1970 Plymouth Hemi Cuda
3,Herkku Gifts,1969 Dodge Charger
4,Herkku Gifts,1993 Mazda RX-7
5,Herkku Gifts,1965 Aston Martin DB5
6,Herkku Gifts,1948 Porsche 356-A Roadster
7,Herkku Gifts,1995 Honda Civic
8,Herkku Gifts,1999 Indy 500 Monte Carlo SS
9,Herkku Gifts,1992 Ferrari 360 Spider red


## 2

In [28]:
c = Customer

num2 = s.query(c.country, func.count(c.customerNumber)).group_by(c.country).filter(or_(c.country == 'Denmark', c.country == 'Norway', c.country == 'Sweden'))
df2 = to_df(num2)
print(sum(df2['count_1']))

7


,country,count_1
0,Denmark,2
1,Norway,3
2,Sweden,2


In [25]:
c = Customer

num2 = s.query(c).filter(or_(c.country == 'Denmark', c.country == 'Norway', c.country == 'Sweden')).count()

print(num2)

7


In [24]:
num2 = session.query(Customer).filter(or_(Customer.country == 'Denmark', Customer.country == 'Norway', Customer.country == 'Sweden')).count()
print(f"Report the number of customers in Denmark, Norway, and Sweden is {num2}")

Report the number of customers in Denmark, Norway, and Sweden is 7
